In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
import pandas as pd

df = pd.read_csv("hf://datasets/sujet-ai/Sujet-Finance-Instruct-177k/sujet-ai/Sujet-Finance-Instruct-177k.csv")

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177597 entries, 0 to 177596
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0       177597 non-null  int64  
 1   inputs           177597 non-null  object 
 2   answer           177597 non-null  object 
 3   system_prompt    177597 non-null  object 
 4   user_prompt      177597 non-null  object 
 5   task_type        177597 non-null  object 
 6   dataset          177597 non-null  object 
 7   index_level      3163 non-null    float64
 8   conversation_id  3163 non-null    float64
dtypes: float64(2), int64(1), object(6)
memory usage: 12.2+ MB


In [8]:
df = df.drop(columns=['dataset', 'Unnamed: 0', 'index_level', 'conversation_id', 'dataset'])

In [16]:
df

,inputs,answer,system_prompt,user_prompt,task_type
0,You are a financial sentiment analysis expert....,neutral,You are a financial sentiment analysis expert....,"According to Gran , the company has no plans t...",sentiment_analysis
1,You are a financial sentiment analysis expert....,neutral,You are a financial sentiment analysis expert....,Technopolis plans to develop in stages an area...,sentiment_analysis
2,You are a financial sentiment analysis expert....,negative,You are a financial sentiment analysis expert....,The international electronic industry company ...,sentiment_analysis
3,You are a financial sentiment analysis expert....,positive,You are a financial sentiment analysis expert....,With the new production plant the company woul...,sentiment_analysis
4,You are a financial sentiment analysis expert....,positive,You are a financial sentiment analysis expert....,According to the company 's updated strategy f...,sentiment_analysis
...,...,...,...,...,...
177592,You are a financial analyst categorizing tweet...,Treasuries | Corporate Debt,You are a financial analyst categorizing tweet...,KfW credit line for Uniper could be raised to ...,topic_classification
177593,You are a financial analyst categorizing tweet...,Treasuries | Corporate Debt,You are a financial analyst categorizing tweet...,KfW credit line for Uniper could be raised to ...,topic_classification
177594,You are a financial analyst categorizing tweet...,Treasuries | Corporate Debt,You are a financial analyst categorizing tweet...,Russian https://t.co/R0iPhyo5p7 sells 1 bln r...,topic_classification
177595,You are a financial analyst categorizing tweet...,Treasuries | Corporate Debt,You are a financial analyst categorizing tweet...,Global ESG bond issuance posts H1 dip as supra...,topic_classification


In [12]:
task_types = df['task_type'].unique()
datasets = {}
for task in task_types:
    datasets[task] = df[df['task_type'] == task]

In [17]:
target_rows = {
    'sentiment_analysis': 5000,
    'yes_no_question': 3000,
    'qa_conversation': 15000,
    'qa': 15000,
    'qa_with_context': 15000,
    'ner_sentiment_analysis': 5000,
    'topic_classification': 12000
}
reduced_dataset = []
for task, data in datasets.items():
    target_size = target_rows.get(task, len(data))

    if len(data) > target_size:
        data = data.sample(n=target_size, random_state=42)

    data = data.drop(columns=['task_type'])

    reduced_dataset.append(data)
final_dataset = pd.concat(reduced_dataset, ignore_index=True)

In [18]:
final_dataset

,inputs,answer,system_prompt,user_prompt
0,You are a financial sentiment analysis expert....,neutral,You are a financial sentiment analysis expert....,Storengy is the GDF SUEZ company that is dedic...
1,You are a financial sentiment analysis expert....,positive,You are a financial sentiment analysis expert....,Wednesdays report adds to evidence of a gradua...
2,You are a financial sentiment analysis expert....,bullish,You are a financial sentiment analysis expert....,Here's Why Renewable Energy Group Shares Jumpe...
3,You are a financial sentiment analysis expert....,neutral,You are a financial sentiment analysis expert....,The short interest contrasts with Handelsbanke...
4,You are a financial sentiment analysis expert....,neutral,You are a financial sentiment analysis expert....,"Of course , you 'll have direct access to Noki..."
...,...,...,...,...
65957,You are a financial analyst categorizing tweet...,Financials,You are a financial analyst categorizing tweet...,"$LWAY - Lifeway Foods GAAP EPS of -$0.01, reve..."
65958,You are a financial analyst categorizing tweet...,Personnel Change,You are a financial analyst categorizing tweet...,"Athenian Group Hires New Director, Bryce Crame..."
65959,You are a financial analyst categorizing tweet...,Markets,You are a financial analyst categorizing tweet...,"Stocks ended lower Tuesday, losing ground in a..."
65960,You are a financial analyst categorizing tweet...,Macro,You are a financial analyst categorizing tweet...,FED BEIGE BOOK :SUBSTANTIAL PRICE INCREASES WE...


In [19]:
data = []
for index, row in final_dataset.iterrows():
    message = {
        "messages": [
            {
                "role": "system",
                "content": row['system_prompt']
            },
            {
                "role": "user",
                "content": row['user_prompt']
            },
            {
                "role": "model",
                "content": row['answer']
            }
        ]
    }
    data.append(message)

In [20]:
data

[{'messages': [{'role': 'system',
    'content': 'You are a financial sentiment analysis expert. Your task is to analyze the sentiment expressed in the given financial text. Only reply with positive, neutral, negative.'},
   {'role': 'user',
    'content': 'Storengy is the GDF SUEZ company that is dedicated to the underground storage of natural gas .'},
   {'role': 'model', 'content': 'neutral'}]},
 {'messages': [{'role': 'system',
    'content': 'You are a financial sentiment analysis expert. Your task is to analyze the sentiment expressed in the given financial text.Only reply with positive, neutral, or negative.'},
   {'role': 'user',
    'content': 'Wednesdays report adds to evidence of a gradual cooling in demand for workers.'},
   {'role': 'model', 'content': 'positive'}]},
 {'messages': [{'role': 'system',
    'content': 'You are a financial sentiment analysis expert. Your task is to analyze the sentiment expressed in the given financial text.Only reply with bearish, neutral, or

In [22]:
import json

In [23]:
with open('final_dataset.jsonl', 'w') as f:
    for item in data:
        json.dump(item, f)
        f.write("\n")


In [31]:
input_file_path1 = 'combined.jsonl'
input_file_path2 = 'final_dataset.jsonl'
output_file_path = 'combined_final.jsonl'

def combine_jsonl_files(file1, file2, output_file):
    with open(file1, 'r', encoding='utf-8') as f1, open(file2, 'r', encoding='utf-8') as f2, open(output_file, 'w', encoding='utf-8') as out:
        for line in f1:
            line = line.strip()  # Remove any surrounding whitespace
            if line:  # Skip empty lines
                try:
                    json.loads(line)  # Validate JSON
                    out.write(line + '\n')
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON in file {file1}: {e}")
        
        for line in f2:
            line = line.strip()  # Remove any surrounding whitespace
            if line:  # Skip empty lines
                try:
                    json.loads(line)  # Validate JSON
                    out.write(line + '\n')
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON in file {file2}: {e}")

    print(f"Combined JSONL file has been saved to {output_file}")

# Combine the JSONL files
combine_jsonl_files(input_file_path1, input_file_path2, output_file_path)

Combined JSONL file has been saved to combined_final.jsonl
